In [ ]:
# ==============================
# Streamlit + Ngrok (with authtoken) in Colab
# ==============================

!pkill ngrok

!pip install streamlit pyngrok joblib scikit-learn xgboost -q

# ---- SETUP NGROK ----
from pyngrok import ngrok
NGROK_AUTH = "32pdo47nOUrEhMkM3qkBKiH3cR1_5HAoYmPj6zfA5t4Km1JKG"   # 🔑 paste your token here
!ngrok config add-authtoken $NGROK_AUTH

# ---- Imports ----
import streamlit as st
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from google.colab import drive

# ---- Mount Google Drive ----
drive.mount('/content/drive')

# Update these paths to your Drive files
MODEL_PATH = "/content/drive/MyDrive/Telecom Customer Churn Prediction/rf_model.pkl"
THRESHOLD_PATH = "/content/drive/MyDrive/Telecom Customer Churn Prediction/best_threshold.pkl"

# Load model & threshold
rf_model = joblib.load(MODEL_PATH)
best_threshold = joblib.load(THRESHOLD_PATH)

# ---- Write Streamlit App ----
with open("app.py", "w") as f:
    f.write("""
import streamlit as st
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler

# Load model & threshold
MODEL_PATH = "/content/drive/MyDrive/Telecom Customer Churn Prediction/rf_model.pkl"
THRESHOLD_PATH = "/content/drive/MyDrive/Telecom Customer Churn Prediction/best_threshold.pkl"

rf_model = joblib.load(MODEL_PATH)
best_threshold = joblib.load(THRESHOLD_PATH)

# Feature engineering function (same as training)
def preprocess_input(data):
    df = pd.DataFrame([data])

    # Binary mapping
    binary_cols = ["gender", "SeniorCitizen", "Partner", "Dependents",
                   "PhoneService", "MultipleLines", "OnlineSecurity", "OnlineBackup",
                   "DeviceProtection", "TechSupport", "StreamingTV", "StreamingMovies",
                   "PaperlessBilling"]
    for col in binary_cols:
        if col in df:
            df[col] = df[col].map({"Yes": 1, "No": 0, "Male": 1, "Female": 0})

    # One-hot encoding (same as training, drop_first=True)
    df = pd.get_dummies(df, columns=["InternetService", "Contract", "PaymentMethod"],
                        drop_first=True, dtype=int)

    # Scale numeric features
    scaler = StandardScaler()
    numeric_cols = ["tenure", "MonthlyCharges", "TotalCharges"]
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

    # Ensure all expected columns exist (missing dummies = 0)
    expected_cols = rf_model.feature_names_in_
    for col in expected_cols:
        if col not in df.columns:
            df[col] = 0
    df = df[expected_cols]  # reorder to match training

    return df

def main():
    st.set_page_config(layout="wide")
    st.title("📊 Customer Churn Prediction App")
    st.write("Predict whether a customer is likely to churn.")

    col1, col2, col3, col4 = st.columns(4)

    with col1:
        tenure = st.number_input("Tenure (months)", min_value=0, max_value=100, value=12)
        gender = st.selectbox("Gender", ["Male", "Female"])
        senior = st.selectbox("Senior Citizen", ["Yes", "No"])
        partner = st.selectbox("Partner", ["Yes", "No"])
        dependents = st.selectbox("Dependents", ["Yes", "No"])

    with col2:
        phone = st.selectbox("Phone Service", ["Yes", "No"])
        multiple = st.selectbox("Multiple Lines", ["Yes", "No"])
        online_sec = st.selectbox("Online Security", ["Yes", "No"])
        online_bkp = st.selectbox("Online Backup", ["Yes", "No"])
        device_prot = st.selectbox("Device Protection", ["Yes", "No"])

    with col3:
        tech = st.selectbox("Tech Support", ["Yes", "No"])
        stream_tv = st.selectbox("Streaming TV", ["Yes", "No"])
        stream_movies = st.selectbox("Streaming Movies", ["Yes", "No"])
        paperless = st.selectbox("Paperless Billing", ["Yes", "No"])
        internet = st.selectbox("Internet Service", ["DSL", "Fiber optic", "No"])

    with col4:
        contract = st.selectbox("Contract", ["Month-to-month", "One year", "Two year"])
        payment = st.selectbox("Payment Method", ["Electronic check", "Mailed check",
                                                  "Bank transfer (automatic)",
                                                  "Credit card (automatic)"])
        monthly_charges = st.number_input("Monthly Charges", min_value=0.0, max_value=200.0, value=70.0)
        total_charges = st.number_input("Total Charges", min_value=0.0, max_value=10000.0, value=1000.0)

    # Collect data
    data = {
        "gender": gender,
        "SeniorCitizen": senior,
        "Partner": partner,
        "Dependents": dependents,
        "PhoneService": phone,
        "MultipleLines": multiple,
        "OnlineSecurity": online_sec,
        "OnlineBackup": online_bkp,
        "DeviceProtection": device_prot,
        "TechSupport": tech,
        "StreamingTV": stream_tv,
        "StreamingMovies": stream_movies,
        "PaperlessBilling": paperless,
        "InternetService": internet,
        "Contract": contract,
        "PaymentMethod": payment,
        "tenure": tenure,
        "MonthlyCharges": monthly_charges,
        "TotalCharges": total_charges
    }

    if st.button("Predict"):
        df_processed = preprocess_input(data)
        prob = rf_model.predict_proba(df_processed)[:,1][0]
        pred = 1 if prob >= best_threshold else 0

        st.subheader("🔮 Prediction Result")
        st.write(f"Churn Probability: **{prob:.2f}**")

        if pred == 1:
            st.error("⚠️ Customer is **likely to churn**")
        else:
            st.success("✅ Customer is **not likely to churn**")

if __name__ == "__main__":
    main()
    """)

# ---- Kill any running Streamlit ----
!pkill streamlit || echo "No Streamlit running"

# ---- Start Streamlit with ngrok tunnel ----
public_url = ngrok.connect(8501)
print("🌍 Public URL:", public_url)
!streamlit run app.py --server.port 8501 &>/dev/null&

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🌍 Public URL: NgrokTunnel: "https://655af0f97566.ngrok-free.app" -> "http://localhost:8501"
